## Rolling 97.5% ES calculation

In [2]:
import pandas as pd
import numpy as np
import os
from scipy.stats import genpareto, chi2
from arch import arch_model
from tqdm import tqdm
import matplotlib.pyplot as plt
import joblib # Needed to load the HMM model
import warnings
warnings.filterwarnings('ignore') # Suppress GARCH warnings

# --- 1. Setup & Load ALL Necessary Data ---
DATA_DIR = "data"
MODEL_DIR = os.path.join("src", "models") # Assuming models are saved here
WINDOW = 252
ES_ALPHA = 0.025 # Corresponds to 97.5% ES

# Load portfolio returns
returns_df = pd.read_csv(
    os.path.join(DATA_DIR, "portfolio_log_returns.csv"),
    index_col='Date',
    parse_dates=True
)
portfolio_returns = returns_df['EqualWeightPortfolio']

# Load Phase 1 results
phase1_results = pd.read_csv(
    os.path.join(DATA_DIR, "phase1_all_risk_measures.csv"),
    index_col='Date',
    parse_dates=True
)
hist_es = phase1_results['HistES_97.5'].rename('Historical_ES_97.5')
try:
    param_t_es = phase1_results['ParamES_T_97.5'].rename('ParamT_ES_97.5')
except KeyError:
    print("Warning: Parametric T ES not found in Phase 1 results. Skipping.")
    param_t_es = None

# --- FIX: Load the saved HMM model ---
try:
    hmm_model_path = os.path.join(MODEL_DIR, "final_hmm_model.joblib")
    final_hmm_model = joblib.load(hmm_model_path)
    print("✅ Loaded saved HMM model.")

    # Recreate features for HMM prediction
    features = pd.DataFrame(index=portfolio_returns.index)
    features['returns'] = portfolio_returns
    features['volatility'] = portfolio_returns.rolling(22).std()
    features.dropna(inplace=True)
    hmm_features = features.values
    
    # Recalculate probabilities
    all_regime_probabilities = final_hmm_model.predict_proba(hmm_features)
    prob_df = pd.DataFrame(all_regime_probabilities, index=features.index)

    # Use the calibrated weights from Phase 4
    calm_weight = 0.65
    nervous_weight = 0.80
    volatile_weight = 0.90
    crisis_weight = 1.00

    volatilities = final_hmm_model.means_[:, 1]
    regime_order = np.argsort(volatilities)
    calm_idx, nervous_idx, volatile_idx, crisis_idx = regime_order

    gjr_weights = np.zeros(4)
    gjr_weights[calm_idx] = calm_weight
    gjr_weights[nervous_idx] = nervous_weight
    gjr_weights[volatile_idx] = volatile_weight
    gjr_weights[crisis_idx] = crisis_weight
    
    # Calculate the weight on GJR for each day
    daily_gjr_weight = prob_df.dot(gjr_weights)
    hmm_loaded_successfully = True

except FileNotFoundError:
    print(f"Error: Saved HMM model not found at {hmm_model_path}. Cannot calculate hybrid ES.")
    daily_gjr_weight = None
    hmm_loaded_successfully = False


# Load static EVT VaR/ES parameters
evt_threshold = 0.0158
evt_shape = 0.1571
evt_scale = 0.0085
losses = -portfolio_returns.dropna()
n = len(losses)
exceedances_for_nu = losses[losses > evt_threshold]
n_u = len(exceedances_for_nu)
var_975_for_es_static = evt_threshold + (evt_scale / evt_shape) * ((n / n_u * (1 - (1 - ES_ALPHA)))**(-evt_shape) - 1)
evt_es_975_static = -(var_975_for_es_static + evt_scale - evt_shape * evt_threshold) / (1 - evt_shape)

print("✅ Data loading complete.")


# --- 2. Define Helper Functions ---
def fit_gjr_garch_model(returns_window): # GJR GARCH fitting
    try:
        model = arch_model(returns_window * 100, vol='Garch', p=1, o=1, q=1, mean='Constant', dist='t')
        fit = model.fit(disp='off')
        if not fit.convergence_flag: return fit, True
        else: return None, False
    except Exception: return None, False

def calculate_fhs_es(window_returns, garch_fit_obj, alpha=ES_ALPHA): # FHS ES
    if garch_fit_obj is None:
        var_thresh = window_returns.quantile(alpha)
        return window_returns[window_returns < var_thresh].mean()
    forecast = garch_fit_obj.forecast(horizon=1)
    cond_mean = forecast.mean.iloc[-1, 0] / 100
    cond_vol = np.sqrt(forecast.variance.iloc[-1, 0]) / 100
    std_resid = garch_fit_obj.resid / garch_fit_obj.conditional_volatility
    var_quantile_resid = std_resid.quantile(alpha)
    es_resid = std_resid[std_resid < var_quantile_resid].mean()
    es_975 = cond_mean + cond_vol * es_resid
    return es_975

# --- 3. Calculate Rolling ES for GJR-GARCH ---
gjr_es_results = []
gjr_dates = []
print("\nCalculating rolling GJR-GARCH ES (97.5%)...")
for i in tqdm(range(WINDOW, len(portfolio_returns))):
    window = portfolio_returns.iloc[i-WINDOW:i]
    current_date = portfolio_returns.index[i]
    gjr_fit, success = fit_gjr_garch_model(window)
    if success:
        es_val = calculate_fhs_es(window, gjr_fit, alpha=ES_ALPHA)
    else:
        var_thresh = window.quantile(ES_ALPHA)
        es_val = window[window < var_thresh].mean()
    gjr_es_results.append(es_val)
    gjr_dates.append(current_date)
gjr_es_975 = pd.Series(gjr_es_results, index=gjr_dates, name='GJR_ES_97.5')

# --- 4. Calculate Hybrid ES ---
hybrid_es_975 = pd.Series(dtype=float, name='Hybrid_ES_97.5') # Initialize
if hmm_loaded_successfully:
    hybrid_es_df = pd.DataFrame(index=daily_gjr_weight.index)
    hybrid_es_df['GJR_Weight'] = daily_gjr_weight
    hybrid_es_df['ES_GJR'] = gjr_es_975
    hybrid_es_df['ES_EVT_Static'] = evt_es_975_static
    hybrid_es_df.dropna(inplace=True)

    hybrid_es_975 = (
        (1 - hybrid_es_df['GJR_Weight']) * hybrid_es_df['ES_EVT_Static'] +
        hybrid_es_df['GJR_Weight'] * hybrid_es_df['ES_GJR']
    ).rename('Hybrid_ES_97.5')
    print("\n✅ Hybrid ES calculation complete.")
else:
    print("\nSkipping Hybrid ES calculation due to missing HMM model.")


# --- 5. Combine and Plot Key ES Series ---
evt_es_975_static_series = pd.Series(evt_es_975_static, index=gjr_es_975.index, name='EVT_ES_97.5_Static')

all_es_list = [hist_es, gjr_es_975, evt_es_975_static_series, hybrid_es_975]
if param_t_es is not None:
    all_es_list.insert(1, param_t_es)

# Ensure proper alignment before final dropna
all_es_series = pd.concat(all_es_list, axis=1)
all_es_series = all_es_series.dropna() # Drop rows where any model failed

print("\n--- Preview of Key ES Series (97.5%) ---")
print(all_es_series.head())

# Plot for comparison
if not all_es_series.empty:
    fig, ax = plt.subplots(figsize=(15, 8))
    all_es_series.plot(ax=ax)
    ax.set_title('Comparison of 97.5% Expected Shortfall Models', fontsize=16)
    ax.set_ylabel('Daily Return (ES)', fontsize=12)
    ax.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("\nNo data to plot after combining ES series.")

print("\n--- Sub-step 1 Complete ---")

Error: Saved HMM model not found at src\models\final_hmm_model.joblib. Cannot calculate hybrid ES.
✅ Data loading complete.

Calculating rolling GJR-GARCH ES (97.5%)...


100%|██████████| 4210/4210 [01:34<00:00, 44.42it/s]


Skipping Hybrid ES calculation due to missing HMM model.

--- Preview of Key ES Series (97.5%) ---
Empty DataFrame
Columns: [Historical_ES_97.5, ParamT_ES_97.5, GJR_ES_97.5, EVT_ES_97.5_Static, Hybrid_ES_97.5]
Index: []

No data to plot after combining ES series.

--- Sub-step 1 Complete ---
